In [3]:
import pandas as pd

In [10]:
account_master = pd.read_csv('fc_account_master.csv', delimiter=',')
account_master.head()

,account_number,customer_code,product,product_category,acc_open_date,acc_closed_date,active_flag
0,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0
1,02XYZXYZ10015593701,KL0255937,SBA,SBPPS,1999-03-10,NaN,0
2,02XYZXYZ10015593801,KL0255938,SBA,SBPPS,1999-03-10,NaN,0
3,02XYZXYZ10015594801,KL0255948,SBA,SBANU,1999-03-22,NaN,0
4,02XYZXYZ10015597601,KL0255976,SBA,SBPPS,1999-04-22,NaN,0


In [11]:
transaction_base =  pd.read_csv('fc_transaction_base.csv', delimiter=',')
transaction_base.head()

,tran_date,account_number,branch,product,lcy_amount,transaction_code,description1,dc_indicator,is_salary
0,09/06/2020,02XYZXYZ10017529992,15.0,SBA,14500.0,CI,ATM WDL /00463701/NABIL\NABIL,withdraw,0.0
1,14/06/2020,02XYZXYZ10017529992,15.0,SBA,20000.0,CI,ATM WDL /00463701/NABIL\NABIL,withdraw,0.0
2,01/06/2020,02XYZXYZ10017517823,15.0,SBA,4000.0,CI,ATM WDL /NICAV701/\Pharping\KT,withdraw,0.0
3,05/06/2020,02XYZXYZ10017517823,15.0,SBA,500.0,CI,ATM WDL /NICAV701/\Pharping\KT,withdraw,0.0
4,12/06/2020,02XYZXYZ10017517823,15.0,SBA,5000.0,CI,ATM WDL /NICAV701/\Pharping\KT,withdraw,0.0


In [12]:
transaction_base['tran_date'] = pd.to_datetime(transaction_base['tran_date'], format='%d/%m/%Y', errors='coerce')
transaction_base['monthly'] = transaction_base['tran_date'].dt.month
transaction_base[['tran_date', 'monthly']]


,tran_date,monthly
0,2020-06-09,6.0
1,2020-06-14,6.0
2,2020-06-01,6.0
3,2020-06-05,6.0
4,2020-06-12,6.0
...,...,...
45027,2020-08-16,8.0
45028,2020-08-16,8.0
45029,2020-08-16,8.0
45030,2020-08-16,8.0


In [13]:
deposit_mask = transaction_base['dc_indicator'] == 'deposit'
withdraw_mask = transaction_base['dc_indicator'] == 'withdraw'

result = transaction_base.groupby(['account_number', 'monthly']).agg(
    average_deposit=('lcy_amount', lambda x: x[deposit_mask].mean()),
    average_withdraw=('lcy_amount', lambda x: x[withdraw_mask].mean()),
    std_deposit=('lcy_amount', lambda x: x[deposit_mask].std()),
    std_withdraw=('lcy_amount', lambda x: x[withdraw_mask].std()),
    var_deposit=('lcy_amount', lambda x: x[deposit_mask].var()),
    var_withdraw=('lcy_amount', lambda x: x[withdraw_mask].var())
).reset_index()

result = result.fillna(0)

In [14]:
result

,account_number,monthly,average_deposit,average_withdraw,std_deposit,std_withdraw,var_deposit,var_withdraw
0,02XYZXYZ10015339701,6.0,950000.000000,45648.000000,294957.624075,59090.604329,8.700000e+10,3.491700e+09
1,02XYZXYZ10015339701,7.0,436065.845000,40169.091429,336204.707648,49802.888915,1.130336e+11,2.480328e+09
2,02XYZXYZ10015339701,8.0,25856.000000,123633.333333,0.000000,116364.098129,0.000000e+00,1.354060e+10
3,02XYZXYZ10015571501,6.0,900000.000000,3265.285714,0.000000,2144.792664,0.000000e+00,4.600136e+06
4,02XYZXYZ10015571501,7.0,307317.236000,202542.882500,405770.888317,362798.668627,1.646500e+11,1.316229e+11
...,...,...,...,...,...,...,...,...
4977,02XYZXYZ110017507524,7.0,8196.193333,570.515000,8621.115890,785.574421,7.432364e+07,6.171272e+05
4978,02XYZXYZ110017507524,8.0,0.000000,6341.800000,0.000000,9356.342031,0.000000e+00,8.754114e+07
4979,02XYZXYZ110017507568,6.0,2623.500000,24000.000000,0.000000,22627.416998,0.000000e+00,5.120000e+08
4980,02XYZXYZ110017507568,7.0,2134.925000,8.070000,2791.056531,0.000000,7.789997e+06,0.000000e+00


In [16]:
merged_df = pd.merge(account_master, transaction_base, how='inner', on = 'account_number')
merged_df.head()

,account_number,customer_code,product_x,product_category,acc_open_date,acc_closed_date,active_flag,tran_date,branch,product_y,lcy_amount,transaction_code,description1,dc_indicator,is_salary,monthly
0,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0,2020-06-01,17.0,SBA,10000.00,CI,ATM WDL /00483901/NABIL\NABIL,withdraw,0.0,6.0
1,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0,2020-06-06,17.0,SBA,5000.00,CI,MPAY Trf to 1210017508555,withdraw,0.0,6.0
2,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0,2020-06-10,17.0,SBA,170000.00,NR,MERRYNA RAI,deposit,0.0,6.0
3,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0,2020-06-01,17.0,SBA,876.15,CI,POS PUR/50001130/SALEWAYS C,withdraw,0.0,6.0
4,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,NaN,0,2020-06-01,17.0,SBA,2174.00,CI,POS PUR/73197330/BBSM SATDO,withdraw,0.0,6.0


In [18]:
deposit_mask = merged_df['dc_indicator'] == 'deposit'
withdraw_mask = merged_df['dc_indicator'] == 'withdraw'

answer = merged_df.groupby(['account_number', 'monthly', 'customer_code']).agg(
    average_deposit=('lcy_amount', lambda x: x[deposit_mask].mean()),
    average_withdraw=('lcy_amount', lambda x: x[withdraw_mask].mean()),
    std_deposit=('lcy_amount', lambda x: x[deposit_mask].std()),
    std_withdraw=('lcy_amount', lambda x: x[withdraw_mask].std()),
    var_deposit=('lcy_amount', lambda x: x[deposit_mask].var()),
    var_withdraw=('lcy_amount', lambda x: x[withdraw_mask].var())
).reset_index()

answer = answer.fillna(0)
answer.head()

,account_number,monthly,customer_code,average_deposit,average_withdraw,std_deposit,std_withdraw,var_deposit,var_withdraw
0,02XYZXYZ10015339701,6.0,KL0253397,950000.000,45648.000000,294957.624075,59090.604329,8.700000e+10,3.491700e+09
1,02XYZXYZ10015339701,7.0,KL0253397,436065.845,40169.091429,336204.707648,49802.888915,1.130336e+11,2.480328e+09
2,02XYZXYZ10015339701,8.0,KL0253397,25856.000,123633.333333,0.000000,116364.098129,0.000000e+00,1.354060e+10
3,02XYZXYZ10015571501,6.0,KL0255715,900000.000,3265.285714,0.000000,2144.792664,0.000000e+00,4.600136e+06
4,02XYZXYZ10015571501,7.0,KL0255715,307317.236,202542.882500,405770.888317,362798.668627,1.646500e+11,1.316229e+11


In [20]:
answer.to_csv('answer.csv', index=False)